<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebooks/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook progetto baseline

## 1. Introduzione al progetto

### 1.1. Obiettivo del modello

L'obiettivo di questo progetto è quello di sviluppare un modello in grado di prevedere il rating ELO di un giocatore (in particolare del giocatore con i pezzi bianchi) a partire da feature estratte da una sua partita.
Questo modello, che chiameremo modello baseline, sarà poi confrontato con un secondo modello ... [DA COMPLETARE]


### 1.2. Presentazione del dataset

Il dataset che abbiamo utilizzato proviene da Kaggle:
https://www.kaggle.com/datasets/datasnaek/chess/code/data
Contiene 20058 partite di scacchi e contiene:
- id (Stringa alfanumerica identificativa della partita);
- rated (La partita è competitiva e comportarà una variazione dell'ELO del giocatore? True o False);
- created_at (Momento di inizio della partita in unix time);
- last_move_at (Momento di fine della partita in unix time);
- turns (Numero di turni di gioco intesi come coppie di mosse bianco-nero)
- victory_status (Come la partita si è conclusa: Scacco matto, stallo, tempo esaurito...);
- winner (Colore del giocatore che ha vinto la partita o draw (patta));
- increment_code (Minuti assegnati a ciascun giocatore all'inizio della partita  secondi assegnati dopo ciascuna mossa);
- white_id (Nome utente del giocatore bianco);
- white_rating (Punteggio ELO del giocatore bianco);
- black_id (Nome utente del giocatore nero);
- black_rating (Punteggio ELO del giocatore nero);
- moves (Tutte le mosse della partita in notazione scacchistica);
- opening_eco (codice identificativo dell'apertura della partita);
- opening_name (nome dell'apertura);
- opening_ply (singole mosse appartenenti alla fase di apertura della partita);

### 1.3. Importazione del Dataset




In [72]:
###INSERIRE CODICE PER IMPORTAZIONE

In [73]:
## Quando ultimeremo il codice sarebbe meglio non fare montare il drive.

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df=pd.read_csv("/content/drive/MyDrive/games.csv")
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  objec

## 2. Sviluppo del modello

### 2.1. Pulizia dei dati

a)  Rimozione dei duplicati (partite con il medesimo ID)

In [74]:
duplicati = df[df['id'].duplicated()]

if not duplicati.empty:
    print(f"Trovati {len(duplicati)} duplicati")
    df = df.drop_duplicates(subset=['id'])

Trovati 945 duplicati


In [78]:
# Ora possiamo eliminare dal dataframe le colonne che sicuramente
# non ci aiuteranno in nessun modo: Gli ID dei giocatori e della partita,
# terremo l'Opening Eco invece che l'Opening Name in quanto più consistente
# e unificato (variazioni della stessa apertura ricadono sotto la stessa categoria).
# Infine rimuoviamo la colonna delle mosse giocate: ovviamente valutare
# una partita, e di conseguenza il livello dei giocatori è un compito che
# si basa solitamente proprio sulle mosse giocate. Quello che vogliamo
# verificare noi, invece, è l'accuratezza con cui si può predire il livello
# di un giocatore avendo solo il rating dell'avversario e informazioni laterali
# sulla partita.

df.drop(columns=['white_id','black_id','id','opening_name','moves'])

#Ora convertiamo le colonne di inizio e fine partita, che sono in formato Unix time, in formato datetime
#Dopodichè inseriremo una più semplice colonna di durata partita al posto delle due colonne iniziali

df_times = df[['created_at','last_move_at']].copy()

df_times['created_at_dt'] = pd.to_datetime(df_times['created_at']/1000, unit='s', origin='unix')

df_times['last_move_at_dt'] = pd.to_datetime(df_times['last_move_at']/1000, unit='s', origin='unix')

df_times['created_time'] = df_times['created_at_dt'].dt.time
df_times['last_move_time'] = df_times['last_move_at_dt'].dt.time
df_times['duration_seconds'] = (df_times['last_move_at_dt'] - df_times['created_at_dt']).dt.total_seconds()

#Adesso che abbiamo la durata della partita in secondi, inseriamo la colonna nel dataframe e rimuoviamo quelle iniziali

df=df.drop(columns=['created_at','last_move_at'])
df=df.join(df_times['duration_seconds'])

#Ora invece ci occupiamo della colonna 'increment_code', che è della forma "X+Y"
#dove X sono i minuti concessi a ciascun giocatore per giocare le proprie mosse e Y
#è il tempo che gli viene riaggiunto una volta giocata la mossa.
#Vogliamo separare questi due valori e tenerli in colonne separate

#Split di increment_code in due colonne
df[['base_time', 'increment_time']] = (
    df['increment_code'].str.split('+', expand=True).astype(int))

#Rimuoviamo la colonna originale
df = df.drop(columns=['increment_code'])

df.tail(50)

KeyError: "None of [Index(['created_at', 'last_move_at'], dtype='object')] are in the [columns]"

In [76]:
#Ora ci occupiamo di creare dummy per le colonne con valori categoriali: victory_status, winner e opening_eco
colonne_categoriali = ['winner', 'victory_status', 'opening_eco']

df = pd.get_dummies(df, columns=colonne_categoriali, dtype=int, drop_first=False)
df.tail()

,id,rated,turns,white_id,white_rating,black_id,black_rating,moves,opening_name,opening_ply,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
20053,EfqH7VVH,True,24,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,Dutch Defense,2,...,0,0,0,0,0,0,0,0,0,0
20054,WSJDhbPl,True,82,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,Queen's Pawn,2,...,0,0,0,0,0,0,0,0,0,0
20055,yrAas0Kj,True,35,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,Queen's Pawn Game: Mason Attack,3,...,0,0,0,0,0,0,0,0,0,0
20056,b0v4tRyF,True,109,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,Pirc Defense,4,...,0,0,0,0,0,0,0,0,0,0
20057,N8G2JHGG,True,78,jamboger,1235,ffbob,1339,d4 d5 Bf4 Na6 e3 e6 c3 Nf6 Nf3 Bd7 Nbd2 b5 Bd3...,Queen's Pawn Game: Mason Attack,3,...,0,0,0,0,0,0,0,0,0,0


In [77]:
df.head()

,id,rated,turns,white_id,white_rating,black_id,black_rating,moves,opening_name,opening_ply,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
0,TZJHLljE,False,13,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,Slav Defense: Exchange Variation,5,...,0,0,0,0,0,0,0,0,0,0
1,l1NXvwaE,True,16,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,Nimzowitsch Defense: Kennedy Variation,4,...,0,0,0,0,0,0,0,0,0,0
2,mIICvQHh,True,61,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,King's Pawn Game: Leonardis Variation,3,...,0,0,0,0,0,0,0,0,0,0
3,kWKvrqYL,True,61,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,Queen's Pawn Game: Zukertort Variation,3,...,0,0,0,0,0,0,0,0,0,0
4,9tXo1AUZ,True,95,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,Philidor Defense,5,...,0,0,0,0,0,0,0,0,0,0
